# Implementação KEM do NTRU-Prime

## Parameters

In [1]:
def randomprime(i):
    return random_prime(2**i-1,True,2**(i-1))

In [2]:
def verifyW(p, q, w, indice):
    while (2*p < 3*w):
        indice = indice-1
        w = randomprime(indice)
    
    while (q < (16*w + 1)):
        indice = indice-1
        w = randomprime(indice)
    
    return w

In [3]:
l = 8

p = randomprime(l)
q = randomprime(l)
w = randomprime(l)

print p
print q
print w

w = verifyW(p,q,w,l)

print w

RingQ.<x> = PolynomialRing(GF(q),'x')
(x^p-x-1).is_irreducible()

Z = IntegerRing()
Ring  = PolynomialRing(Z,'x')
Ring3 = PolynomialRing(GF(3),'x')

R  = QuotientRing(Ring, Ring.ideal(x^p-x-1))
R3 = QuotientRing(Ring3, Ring3.ideal(x^p-x-1))
Rq = QuotientRing(RingQ, RingQ.ideal(x^p-x-1))

167
233
131
11


## Funções auxiliares

In [4]:
def roundEach(f,b=None):
    ff = list(f)
    if b == None:
        return ff
    else:
        fp = map(lift,[Mod(a,b) for a in ff])
        return [u if u <= b//2 else u-b for u in fp ]

def toR(vec):
    return R(vec)

def computeH(f,g):
    _f = Rq(f)
    _g = Rq(g)
    
    try:
        hq = (_g/_f)
        return (True,Rq([lift(a) for a in list(hq)]))
    except:
        return (False,0)

In [5]:
import random as rn

def small(P=p):
    u = [rn.choice([-1,0,1]) for i in range(p)]
    return u

def smallW(P=p,W=w):
    u = [rn.choice([-1,1]) for i in range(w)] + [0]*(p-w)
    rn.shuffle(u)
    return u

In [6]:
def verifyG():
    while True:
        g = small()
        
        if(R3(g).is_unit()):
            break
            
    return g

def verifyF():
    while True:
        f = smallW()
        
        if(Rq(f).is_unit()):
            break
            
    return f

In [ ]:
def formatC(ciphertext):
    c = ciphertext[33:(size-1)].split(",")
    
    for i in range(len(c)):
        c[i] = int(c[i])
    
    return c

## Key generation, Encapsulate e Decapsulate

In [7]:
import hashlib

def KeyGen(j=q, k=p, l=w):
    while True:
        G = verifyG()
        F = smallW()
        f = 1 + 3 * toR(F)
        g = 3 * toR(G)
        (flag, h) = computeH(f,g)
        if flag:
            _g = R3(G).inverse_of_unit()
            break
    return {'f': R(F), 'g': _g, 'pk' : h} 

def Encapsulate(plaintext,pk):
    Rr = smallW()
    r = toR(Rr)
    m = toR(plaintext)
    c = roundEach(pk*r + m, b=q)
    fhash = hashlib.sha512()
    fhash.update(str(Rr).encode('utf-8'))
    divisao = fhash.digest()
    C = divisao[:32]
    K = divisao[32:]
    c = str(c)
    return {'Cc': C+c, 'K': K}
    
def Decapsulate(ciphertext,f,g):
    C = ciphertext[:32]
    print C
    size = len(ciphertext)
    print ciphertext[32:]
    c = formatC(ciphertext)
    print c
   # e = toR(ciphertext)
   # a = roundEach(sk*e, b=q)
    return 1 #roundEach(toR(a), b=3)

## Test

In [8]:
import base64

def run():
    keys   = KeyGen()
    plain  = small()
    crypto = Encapsulate(plain,keys['pk'])
    decryp = Decapsulate(crypto['Cc'],keys['f'],keys['g'])
    return base64.b64encode(crypto['K'])

In [9]:
run()

��'�Z�,�E�F�E�� L$�R㑾n�%��
[53, -31, -77, -71, 17, 16, -3, -84, -57, -59, 8, 21, -27, 13, 94, -41, 0, -53, -80, 99, 1, -34, 62, -93, -114, -69, -26, 39, 60, 31, -78, -66, -57, -9, -112, 107, 52, 34, -29, 40, 32, -40, 108, -17, 73, 66, 14, 71, 94, 25, -70, 47, -102, -17, 66, 51, -44, 44, -29, 37, -116, -84, -48, 93, -103, -114, 40, 107, -55, 58, 41, -94, 80, -67, -110, -45, 32, 87, 104, 70, 28, 85, 46, 71, 110, 74, 49, 24, 22, 54, -36, 14, 115, -25, 90, -96, -89, 47, -68, 6, 110, 62, 103, 24, 2, -58, 73, 86, 0, -24, 12, 112, -94, -114, 21, -87, 7, 0, 20, -82, -113, 42, 90, 22, 53, -41, 33, 93, 80, 61, -94, 27, -88, -112, 86, 111, 39, -21, -54, -59, 78, -8, 94, 106, -78, 115, -66, -96, 84, -4, -60, -113, -16, -45, -106, 80, -100, 56, 54, -88, -55, 105, -76, 60, -11, 108, -43]
[53, -31, -77, -71, 17, 16, -3, -84, -57, -59, 8, 21, -27, 13, 94, -41, 0, -53, -80, 99, 1, -34, 62, -93, -114, -69, -26, 39, 60, 31, -78, -66, -57, -9, -112, 107, 52, 34, -29, 40, 32, -40, 108, -17, 73, 66, 14, 

'6ZFlLrSTSq10hxg0LhPakigC3PXCyeLMKGLlkXCvQUc='